In [3]:
import functools

class Grammar:
    def __init__(self, tokens, rules, ssymbol):
        self.tokens  = tokens
        self.rules   = rules
        self.ssymbol = ssymbol

    def __str__(self):
        def sentence_to_str(sentence):
            tostr = lambda symbol : symbol if isinstance(symbol,str) else str(symbol[1])
            return functools.reduce(lambda s1,s2: tostr(s1)+tostr(s2), sentence,"")
                    
        grammar = ""
        for nonterminal, sentences in self.rules.items():
            grammar += f"{nonterminal} -> {functools.reduce(lambda s1,s2 : sentence_to_str(s1)+' | '+sentence_to_str(s2), sentences)}\n"

        return grammar

    def first(self, symbol):
        terminals = set()

        if symbol in self.tokens:
            terminals.add(symbol)
        else:
            for sentence in self.rules[symbol]:
                terminals|= self.first_of_sentence(sentence)

        return terminals

    def first_of_sentence(self, sentence):
        terminals = set()

        for idx, symbol in enumerate(sentence):
            fs = self.first(symbol)

            if ( (0,'e') in fs and idx+1 < len(sentence)):
                terminals |= fs - {(0,'e')}
            else:
                terminals |= fs
                break

        return terminals

    def follow(self, symbol):
        terminals = set()

        if (symbol == 'S'):
            terminals.add('$')
        
        for nonterminal, sentences in self.rules.items():
            for sentence in sentences:
                for idx, s in enumerate(sentence):
                    if (symbol == s):
                        # case of 'aB'
                        if ( idx+1 >= len(sentence)):
                            terminals |= self.follow(nonterminal)
                        # cases of 'aBc' where 'a' and 'b' are sentences
                        else:
                            fs = self.first_of_sentence(sentence[idx+1:len(sentence)])
                            
                            # if the follow part of sentence is vanishing then find follow set of the left-nonterminal
                            if ( (0,'e') in fs): 
                                terminals |= fs - {(0,'e')} | self.follow(nonterminal)
                            else:
                                terminals |= fs
        
        return terminals

In [6]:
def main():
    grammar = Grammar(
        {
            (0,'e'),
            (1,'a'),
            (2,'b'),
            (3,'c'),
            (4,'d'),
            (5,'f'),
            (6,'g'),
            (7,'h')
        },
        {
            'A' : [
                [(4,'d'), (1,'a')],
                ['B', 'C']
            ],
            'B' : [
                [(6,'g')],
                [(0,'e')]
            ],
            'C' : [
                [(7,'h')],
                [(0,'e')]
            ],
            'S' : [
                ['A','C','B'],
                ['C', (2,'b'),(2,'b')],
                ['B', (1,'a')]
            ]
        },
        'S'
    )

    print(f"Grammar :\n{grammar}")


    for nonterminal in grammar.rules:
       print(f'First({nonterminal}) -- {grammar.first(nonterminal)}')

    print()

    for nonterminal in grammar.rules:
       print(f'Follow({nonterminal}) -- {grammar.follow(nonterminal)}')
    

if __name__ == "__main__":
    main()

Grammar :
A -> da | BC
B -> g | e
C -> h | e
S -> ACB | Cbb | Ba

First(A) -- {(0, 'e'), (6, 'g'), (7, 'h'), (4, 'd')}
First(B) -- {(0, 'e'), (6, 'g')}
First(C) -- {(0, 'e'), (7, 'h')}
First(S) -- {(0, 'e'), (6, 'g'), (7, 'h'), (2, 'b'), (4, 'd'), (1, 'a')}

Follow(A) -- {'$', (7, 'h'), (6, 'g')}
Follow(B) -- {'$', (7, 'h'), (6, 'g'), (1, 'a')}
Follow(C) -- {'$', (6, 'g'), (7, 'h'), (2, 'b')}
Follow(S) -- {'$'}
